In [1]:
import pandas as pd
import numpy as np

In [2]:
test_1 = pd.read_csv("CMaps/test_FD001.txt",sep=" ",header=None)
train_1 = pd.read_csv("CMaps/train_FD001.txt",sep=" ",header=None)

In [5]:
columns = ['engine', 'cycle', 'setting1', 'setting2', 'setting3', 'sensor1',
       'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6', 'sensor7',
       'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 'sensor13',
       'sensor14', 'sensor15', 'sensor16', 'sensor17', 'sensor18', 'sensor19',
       'sensor20', 'sensor21','sensor22','sensor23' ]
test_1.columns = columns
train_1.columns = columns

In [6]:
train_1.drop(['sensor22','sensor23'], axis=1, inplace=True)

test_1.drop(['sensor22','sensor23'], axis=1, inplace=True)

In [8]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [12]:
def Calculate_RUL(df):
    max_cycles = df.groupby('engine')['cycle'].max()
    merged = df.merge(max_cycles.to_frame(name='max_time_cycle'), left_on='engine',right_index=True)
    merged["RUL"] = merged["max_time_cycle"] - merged['cycle']
    merged = merged.drop("max_time_cycle", axis=1)
    return merged
train_1 = Calculate_RUL(train_1)


In [13]:
X1=train_1.drop(columns=['engine', 'cycle', 'setting3', 'sensor1', 'sensor5','sensor16','sensor18','sensor19' ,'RUL'])
y1=train_1['RUL']
X_train1,X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.20, random_state=42)
rul1=pd.read_csv("CMaps/RUL_FD001.txt",header=None, names=["RUL"])


In [14]:
scaled_train1=scaler.fit_transform(X_train1)
scaled_test1=scaler.fit_transform(X_test1)

X_valid1 = test_1.groupby('engine').last().reset_index().drop(columns=['engine', 'cycle', 'setting3', 'sensor1', 'sensor5','sensor16','sensor18','sensor19'])

scaled_X_valid1=scaler.fit_transform(X_valid1)

In [15]:
from sklearn.decomposition import PCA

# PCA for dimensionality reduction
pca = PCA(n_components=5)
X_pca = pca.fit_transform(scaled_X_valid1)